In [1]:
import os

# Get the current working directory
cwd = os.getcwd()
cwd

'/notebooks/pll_data_detection/exp'

In [126]:
EXP_NAME = "exp001"
MODEL_NAME = "distilbert-base-uncased"
LOG_PATH = f"pll_data_detection/log/{EXP_NAME}"
MODEL_OUTPUT_PATH = f"pll_data_detection/trained_models/{EXP_NAME}"
SUB_OUTPUT_PATH = f"pll_data_detection/submissions/{EXP_NAME}"

# Install

In [132]:
%pip install evaluate

6346.12s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install polars
%pip install transformers==4.37.2
%pip install datasets==2.16.1
%pip install evaluate==0.4.1
%pip install seqeval==1.2.2

# formatter
%pip install black isort

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.8/26.8 MB 43.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 33.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of aiohttp to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fsspec[http] to determi

# Import

In [134]:
import os

import evaluate
import numpy as np
import polars as pl
import torch
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TokenClassificationPipeline,
    Trainer,
    TrainingArguments,
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [135]:
import transformers

transformers.__version__

'4.21.3'

In [136]:
import datasets

datasets.__version__

'2.16.1'

In [138]:
import seqeval

seqeval.__version__

AttributeError: module 'seqeval' has no attribute '__version__'

# Check Environment

In [6]:
!python --version

Python 3.9.16


In [7]:
!nvidia-smi

Sun Feb  4 00:42:34 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 26%   30C    P8     6W / 180W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Data Load

In [8]:
# データ確認用
train = pl.read_json("pll_data_detection/train.json")
train.head()

document,full_text,tokens,trailing_whitespace,labels
i64,str,list[str],list[bool],list[str]
7,"""Design Thinkin…","[""Design"", ""Thinking"", … "" ""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
10,"""Diego Estrada …","[""Diego"", ""Estrada"", … "" ""]","[true, false, … false]","[""B-NAME_STUDENT"", ""I-NAME_STUDENT"", … ""O""]"
16,"""Reporting proc…","[""Reporting"", ""process"", … "" ""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
20,"""Design Thinkin…","[""Design"", ""Thinking"", … "" ""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
56,"""Assignment: V…","[""Assignment"", "":"", … "" ""]","[false, false, … false]","[""O"", ""O"", … ""O""]"


In [9]:
# データ確認用
test = pl.read_json("pll_data_detection/test.json")
test

document,full_text,tokens,trailing_whitespace
i64,str,list[str],list[bool]
7,"""Design Thinkin…","[""Design"", ""Thinking"", … "" ""]","[true, true, … false]"
10,"""Diego Estrada …","[""Diego"", ""Estrada"", … "" ""]","[true, false, … false]"
16,"""Reporting proc…","[""Reporting"", ""process"", … "" ""]","[true, false, … false]"
20,"""Design Thinkin…","[""Design"", ""Thinking"", … "" ""]","[true, true, … false]"
56,"""Assignment: V…","[""Assignment"", "":"", … "" ""]","[false, false, … false]"
86,"""Cheese Startup…","[""Cheese"", ""Startup"", … "" ""]","[true, true, … false]"
93,"""Silvia Villalo…","[""Silvia"", ""Villalobos"", … "" ""]","[true, false, … false]"
104,"""Storytelling …","[""Storytelling"", "" "", … "" ""]","[true, false, … false]"
112,"""Reflection – L…","[""Reflection"", ""–"", … "" ""]","[true, true, … false]"


In [10]:
train_dataset = load_dataset(
    "json", data_files={"train": "pll_data_detection/train.json"}, split="train"
)
test_dataset = load_dataset(
    "json", data_files={"test": "pll_data_detection/test.json"}, split="test"
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
# debug
train_dataset = train_dataset.select(range(100))

In [12]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [13]:
# TODO: 512?が上限であるのに対し、NER対象のテキストが512よりも長い場合が多いため、適したモデルにする
# DeBERTaあたりは、512を超えるテキストに対応していたような
example = train_dataset[0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

Token indices sequence length is longer than the specified maximum sequence length for this model (748 > 512). Running this sequence through the model will result in indexing errors


['[CLS]',
 'design',
 'thinking',
 'for',
 'innovation',
 'reflex',
 '##ion',
 '-',
 'av',
 '##ril',
 '2021',
 '-',
 'nat',
 '##hal',
 '##ie',
 'sy',
 '##lla',
 'challenge',
 '&',
 'selection',
 'the',
 'tool',
 'i',
 'use',
 'to',
 'help',
 'all',
 'stakeholders',
 'finding',
 'their',
 'way',
 'through',
 'the',
 'complexity',
 'of',
 'a',
 'project',
 'is',
 'the',
 'mind',
 'map',
 '.',
 'what',
 'exactly',
 'is',
 'a',
 'mind',
 'map',
 '?',
 'according',
 'to',
 'the',
 'definition',
 'of',
 'bu',
 '##zan',
 't',
 '.',
 'and',
 'bu',
 '##zan',
 'b',
 '.',
 '(',
 '1999',
 ',',
 'des',
 '##sin',
 '##e',
 '-',
 'moi',
 'l',
 "'",
 'intelligence',
 '.',
 'paris',
 ':',
 'les',
 'editions',
 'd',
 "'",
 'organisation',
 '.',
 ')',
 ',',
 'the',
 'mind',
 'map',
 '(',
 'or',
 'he',
 '##uri',
 '##stic',
 'diagram',
 ')',
 'is',
 'a',
 'graphic',
 'representation',
 'technique',
 'that',
 'follows',
 'the',
 'natural',
 'functioning',
 'of',
 'the',
 'mind',
 'and',
 'allows',
 'the',
 '

In [14]:
# labelを変換する
id2label = {
    0: "O",
    1: "B-NAME_STUDENT",
    2: "I-NAME_STUDENT",
    3: "B-EMAIL",
    4: "I-EMAIL",
    5: "B-USERNAME",
    6: "I-USERNAME",
    7: "B-ID_NUM",
    8: "I-ID_NUM",
    9: "B-PHONE_NUM",
    10: "I-PHONE_NUM",
    11: "B-URL_PERSONAL",
    12: "I-URL_PERSONAL",
    13: "B-STREET_ADDRESS",
    14: "I-STREET_ADDRESS",
}

label2id = {v: k for k, v in id2label.items()}


def label2id_func(example):
    example["labels"] = [label2id[tag] for tag in example["labels"]]
    return example


labele2id_train_dataset = train_dataset.map(label2id_func)

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [15]:
label_list = list(label2id.keys())
label_list

['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-EMAIL',
 'I-EMAIL',
 'B-USERNAME',
 'I-USERNAME',
 'B-ID_NUM',
 'I-ID_NUM',
 'B-PHONE_NUM',
 'I-PHONE_NUM',
 'B-URL_PERSONAL',
 'I-URL_PERSONAL',
 'B-STREET_ADDRESS',
 'I-STREET_ADDRESS']

In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["labels"]):
        # トークン化されたシーケンス内の各トークンが元のテキスト内のどの単語に対応するかを示すIDを提供します。
        # 例えば、元のテキストが "Hello, world!" で、トークン化されたシーケンスが ["Hello", ",", "world", "!"] の場合
        # `word_ids`メソッドは [0, None, 1, None] を返します。
        # これは、"Hello" が最初の単語（インデックス0）、"," が単語に属さない（None）、"world" が2番目の単語（インデックス1）、"!" が単語に属さない（None）ことを示します。
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif (
                word_idx != previous_word_idx
            ):  # subwordの場合、同じword_idxが連続している。最初のtoken以外は-100にする
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_train_dataset = labele2id_train_dataset.map(
    tokenize_and_align_labels, batched=True
)

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [17]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [18]:
seqeval = evaluate.load("seqeval")

In [19]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [20]:
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [21]:
# evalデータの用意
tokenized_train_valid_dataset = tokenized_train_dataset.train_test_split(test_size=0.2)

In [22]:
training_args = TrainingArguments(
    output_dir=LOG_PATH,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_valid_dataset["train"],
    eval_dataset=tokenized_train_valid_dataset["test"],  # 名前はtestだが、実際はvalid
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [24]:
# モデルの保存
# trainer.train()
# trainer.save_model(MODEL_OUTPUT_PATH)

# Inference

In [63]:
def combine_subword(results):
    # サブワードを結合
    combined_results = []
    current_entity = None
    for result in results:
        if result["word"].startswith("##"):
            # サブワードの場合、現在のエンティティに追加
            if current_entity is not None:
                current_entity["word"] += result["word"][2:]
                current_entity["end"] = result["end"]
                current_entity["score"] = (
                    current_entity["score"] + result["score"]
                ) / 2
        else:
            # 新しいエンティティの場合、現在のエンティティを結果に追加
            if current_entity is not None:
                combined_results.append(current_entity)
            current_entity = result

    # 最後のエンティティを結果に追加
    if current_entity is not None:
        combined_results.append(current_entity)

    return combined_results

In [23]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_OUTPUT_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_OUTPUT_PATH)

loading configuration file pll_data_detection/trained_models/exp001/config.json
Model config DistilBertConfig {
  "_name_or_path": "pll_data_detection/trained_models/exp001",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "O",
    "1": "B-NAME_STUDENT",
    "2": "I-NAME_STUDENT",
    "3": "B-EMAIL",
    "4": "I-EMAIL",
    "5": "B-USERNAME",
    "6": "I-USERNAME",
    "7": "B-ID_NUM",
    "8": "I-ID_NUM",
    "9": "B-PHONE_NUM",
    "10": "I-PHONE_NUM",
    "11": "B-URL_PERSONAL",
    "12": "I-URL_PERSONAL",
    "13": "B-STREET_ADDRESS",
    "14": "I-STREET_ADDRESS"
  },
  "initializer_range": 0.02,
  "label2id": {
    "B-EMAIL": 3,
    "B-ID_NUM": 7,
    "B-NAME_STUDENT": 1,
    "B-PHONE_NUM": 9,
    "B-STREET_ADDRESS": 13,
    "B-URL_PERSONAL": 11,
    "B-USERNAME": 5,
    "I-EMAIL": 4,
    "I-ID_NUM": 8,
    "I-NAME_STUDENT": 2,
    "I-

In [33]:
token_classification = TokenClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",  # サブワードを集約して元の単語レベルでの予測を得る
    ignore_subwords=True,  # サブワードの予測を無視する
)

[7, 10, 16, 20, 56, 86, 93, 104, 112, 123]

In [68]:
combined_results

[{'entity': 'B-NAME_STUDENT',
  'score': 0.4579974,
  'index': 29,
  'word': 'stefano',
  'start': 156,
  'end': 163},
 {'entity': 'B-NAME_STUDENT',
  'score': 0.3540913462638855,
  'index': 30,
  'word': 'lovato',
  'start': 164,
  'end': 170},
 {'entity': 'I-NAME_STUDENT',
  'score': 0.32486668,
  'index': 33,
  'word': 'md',
  'start': 172,
  'end': 174}]

In [93]:
test_dataset["document"]

[7, 10, 16, 20, 56, 86, 93, 104, 112, 123]

In [118]:
outputs = []
for document_id, full_text, tokens in tqdm(
    zip(test_dataset["document"], test_dataset["full_text"], test_dataset["tokens"]),
    total=len(test_dataset),
):
    # 予測の実行
    results = token_classification(full_text)
    # サブワードを結合
    combined_results = combine_subword(results)
    # 結合されたサブワードのindexより、元のテキストを取得
    org_texts_entities = [
        (
            full_text[combined_result["start"] : combined_result["end"]],
            combined_result["entity"],
        )
        for combined_result in combined_results
    ]
    # tokensの中から、org_textsに一致するもののindexを取得
    tokens_idx = [(idx, token) for idx, token in enumerate(tokens)]
    used_idx = 0
    for org_text, entity in org_texts_entities:
        for token_idx, token in tokens_idx[used_idx:]:  # 前回のindexより後しか見ない
            if token.startswith(org_text):
                outputs.append([document_id, token_idx, entity, org_text, token])
                used_idx = token_idx
                break

  0%|          | 0/10 [00:00<?, ?it/s]

In [130]:
import polars as pl

output_df = pl.DataFrame(
    outputs, schema=["document", "token", "label", "org_text", "token_text"]
)

os.makedirs(SUB_OUTPUT_PATH, exist_ok=True)
output_df.select(pl.col(["document", "token", "label"])).with_row_index(
    name="row_id"
).write_csv(f"{SUB_OUTPUT_PATH}/sample_sumission.csv")